In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
! pip install pyannote.audio

In [2]:
!pip install pydub

In [14]:
import torch
import os
from pyannote.audio import Pipeline
import pydub

In [15]:
from google.colab import userdata
PYANNOTE_API_KEY = userdata.get("PYANNOTE_API_KEY")
HF_TOKEN = userdata.get("HF_TOKEN")

In [16]:
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HF_TOKEN)

In [17]:
pipeline.to(torch.device("cuda"))


In [18]:
def trim_and_save(source_filename, speaker, beginning,end):

  audio = pydub.AudioSegment.from_wav(source_filename)

  # Define the start and end times (in milliseconds)
  start_time = beginning * 1000  # Replace 'n' with the desired start time in seconds
  end_time = end * 1000    # Replace 'm' with the desired end time in seconds

  # Trim the audio file
  trimmed_audio = audio[beginning:end]

  # Export the trimmed audio to a new file
  trimmed_audio.export(f"{beginning}-{end}:{speaker}.wav", format="wav")


In [19]:
def diarize_locally(filename):
    diarization = pipeline(filename)
    activeSpeaker = -1
    startingPeriod = 0
    endingPeriod = -1
    for turn, _, speaker in diarization.itertracks(yield_label=True):
      # print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

      if speaker != activeSpeaker:
        trim_and_save(filename, activeSpeaker, startingPeriod, endingPeriod)
        startingPeriod = turn.start
        activeSpeaker = speaker
      else:
        endingPeriod = turn.end

      print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

In [21]:
diarize_locally("videoplayback.wav")

start=0.0s stop=0.0s speaker_SPEAKER_01
start=0.1s stop=1.0s speaker_SPEAKER_01
start=1.1s stop=1.8s speaker_SPEAKER_01
start=3.4s stop=4.9s speaker_SPEAKER_00
start=5.4s stop=9.4s speaker_SPEAKER_01
start=9.8s stop=13.1s speaker_SPEAKER_01
start=13.9s stop=18.3s speaker_SPEAKER_01
start=18.8s stop=19.6s speaker_SPEAKER_00
start=20.2s stop=22.3s speaker_SPEAKER_00
start=23.0s stop=26.8s speaker_SPEAKER_00
start=27.3s stop=29.2s speaker_SPEAKER_00
start=29.5s stop=30.1s speaker_SPEAKER_00
start=30.8s stop=31.7s speaker_SPEAKER_00
start=32.0s stop=32.9s speaker_SPEAKER_01
start=33.0s stop=34.1s speaker_SPEAKER_01
start=34.6s stop=36.8s speaker_SPEAKER_00
start=36.8s stop=42.0s speaker_SPEAKER_01


In [22]:
!pwd


/content


In [ ]:
import os
from IPython.display import Audio, display

def play_sounds_from_directory(directory):
    # List all files in the directory
    sound_files = [file for file in os.listdir(directory) if file.endswith('.wav')]

    # Sort files for a consistent playback order
    sound_files.sort()

    # Play each sound file
    for sound_file in sound_files:
        file_path = os.path.join(directory, sound_file)
        if input("Play another one? ") == "y":
          display(Audio(file_path, autoplay=True))
          os.remove()

